In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset('glue', 'sst2')

In [ ]:
pip install transformers

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)
tokenizer.pad_token = tokenizer.eos_token

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1200))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1200))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Validation accuracy: {eval_results:.4f}")